# New Section

RNN Architecture
For the RNN model, long-short term memory (LSTM),
inspired by natural language processing, is one of the most
popular EEG-based MI classification structures. Since raw
EEG signals are in time series, the time domain's correlation can be used for MI class prediction

https://arxiv.org/pdf/2101.10932.pdf


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
dir = "/content/drive/MyDrive/CruX BCI team 3/Colab Notebooks/Datasets/EEG data from hands movement" # Howard
# dir = "/content/drive/MyDrive/Clubs/CruX/CruX/Colab Notebooks/Datasets/EEG data from hands movement" # Marvin
dfs = [pd.read_csv(dir + '/user_' + user + '.csv') for user in ['a', 'b', 'c', 'd']]


Mounted at /content/drive/


In [ ]:
for i in range(len(dfs)):
  dfs[i]['User'] = pd.Series(i, index=dfs[i].index)

In [ ]:
data = pd.concat(dfs, axis=0).sample(frac=1.0, random_state=123).reset_index(drop=True)

In [ ]:
data.head()

,Class,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,...,F8 beta m,AF4 delta std,AF4 delta m,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m,User
0,0.0,3572.252440,2065.056469,0.851824,2.047953,0.651871,2.522036,2.109733,3.347705,3564.779879,...,35.151586,3628.426885,2129.789645,5.353671,17.885132,7.672209,29.960618,43.216980,43.932669,0
1,1.0,3574.116024,2065.528155,1.932513,3.321636,1.138012,2.349805,2.256212,3.945981,3563.399422,...,40.800889,3680.341349,2144.200503,10.819521,36.995982,12.812193,24.146774,23.747501,49.072017,0
2,0.0,3554.487593,2056.215665,0.935015,3.793783,0.736168,2.680542,3.381325,4.678876,3568.839949,...,18.176841,3538.347368,2081.315814,5.486555,13.204753,0.664075,6.633072,1.434277,4.132446,2
3,0.0,3570.668125,2063.974908,1.875394,3.028541,0.910000,3.018672,1.163312,2.883009,3565.403408,...,55.547547,3604.601528,2122.493834,15.611283,16.452483,22.462175,46.703612,32.213578,70.892466,0
4,0.0,3559.747108,2057.401763,1.053691,1.530594,1.593121,2.789907,2.668865,4.326693,3573.651774,...,7.208052,3513.244789,2030.461207,1.455450,3.030659,0.482971,3.200647,0.895170,2.099638,2


In [ ]:
def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [ ]:
def preprocess_inputs(df, target='Class'):
    df = df.copy()
    
    # One-hot encode whichever target column is not being used
    targets = ['Class', 'User']
    targets.remove(target)
    df = onehot_encode(df, column=targets[0])
    
    # Split df into X and y
    y = df[target].copy()
    X = df.drop(target, axis=1)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=50)
    
    # Scale X with a standard scaler
    scaler = StandardScaler()
    scaler.fit(X_train)

    X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
    
    return X_train, X_test, y_train, y_test

In [ ]:
def build_model(num_classes=4):
    
    inputs = tf.keras.Input(shape=(None, 116))
    x = tf.keras.layers.Reshape((-1, 116))(inputs)
    x = tf.keras.layers.LSTM(256, return_sequences=True)(x)
    x = tf.keras.layers.LSTM(128, return_sequences=True)(x)
    x = tf.keras.layers.LSTM(64)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
X_train, X_test, y_train, y_test = preprocess_inputs(data, target='Class')

In [ ]:
X_train

,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,F7 delta m,...,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m,User_0,User_1,User_2,User_3
0,0.136557,0.109199,-0.762146,-0.834782,-1.100221,-0.054621,0.025642,-0.592080,-0.503282,-0.238427,...,-0.453468,-0.459838,-0.381190,-0.451708,-0.508121,-0.532689,-0.576849,1.748749,-0.580524,-0.580190
1,0.055419,-0.097328,-0.187676,-0.538443,-0.824569,-0.541600,-0.763167,-0.789313,-1.258213,-1.347086,...,-0.603038,-0.413381,-0.459061,-0.446980,-0.526168,-0.517365,-0.576849,-0.571837,-0.580524,1.723573
2,0.103683,0.036032,0.507173,0.065498,0.415348,1.354333,1.190130,1.538453,1.379261,1.485966,...,-0.610913,-0.367447,-0.306535,-0.213570,-0.494130,-0.463914,-0.576849,-0.571837,1.722581,-0.580190
3,-1.015328,-0.064868,0.323611,0.054294,0.796912,0.984926,1.053840,1.861944,1.499653,1.849122,...,-0.348525,-0.212232,-0.305695,-0.314834,-0.487564,-0.512801,-0.576849,-0.571837,1.722581,-0.580190
4,-0.239211,-0.186150,-0.515752,0.201501,1.746550,0.395262,-1.252509,-1.468086,-0.431629,-0.680551,...,-0.504640,-0.503606,-0.328398,-0.439712,-0.553785,-0.560519,-0.576849,-0.571837,-0.580524,1.723573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9211,-1.625647,-1.510879,-0.601444,-0.272073,-0.438529,0.602139,2.976946,0.754882,1.895693,2.055473,...,-0.543285,-0.422701,-0.470908,-0.452206,-0.495726,-0.449000,-0.576849,-0.571837,1.722581,-0.580190
9212,-1.064711,-1.203765,-0.348828,-0.398137,-0.074671,0.477677,1.374159,2.214734,1.298586,1.672208,...,-0.607755,-0.500974,-0.265585,-0.420859,-0.463723,-0.454791,-0.576849,-0.571837,1.722581,-0.580190
9213,-0.163182,-0.221880,-0.504956,0.542650,1.029620,-0.089580,-1.289843,-1.078598,-1.139330,-1.282820,...,-0.386812,-0.302039,-0.395470,-0.510833,-0.538754,-0.552641,-0.576849,-0.571837,-0.580524,1.723573
9214,-0.186939,-0.433750,0.894462,-0.172772,0.014341,0.055100,-0.553529,-0.989989,0.229215,-0.100149,...,-0.435756,-0.514580,-0.497519,-0.518822,-0.517426,-0.561628,-0.576849,1.748749,-0.580524,-0.580190


In [ ]:
y_train

10812    2.0
9395     2.0
1667     1.0
11267    1.0
5678     2.0
        ... 
8324     2.0
10206    1.0
6253     2.0
10123    0.0
5600     2.0
Name: Class, Length: 9216, dtype: float64

In [ ]:
y_train.value_counts()

1.0    3098
0.0    3073
2.0    3045
Name: Class, dtype: int64

In [ ]:
# try 1st set of parameters
class_model = build_model(num_classes=4)

class_history = class_model.fit(
    X_train,
    y_train,
    validation_split=0.4,
    batch_size=10,
    epochs=12,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/12
553/553 [==============================] - 32s 39ms/step - loss: 1.1070 - accuracy: 0.3682 - val_loss: 1.0528 - val_accuracy: 0.4302
Epoch 2/12
553/553 [==============================] - 15s 27ms/step - loss: 1.0307 - accuracy: 0.4565 - val_loss: 1.0110 - val_accuracy: 0.4771
Epoch 3/12
553/553 [==============================] - 14s 26ms/step - loss: 0.9644 - accuracy: 0.5109 - val_loss: 0.9628 - val_accuracy: 0.4909
Epoch 4/12
553/553 [==============================] - 15s 28ms/step - loss: 0.8957 - accuracy: 0.5610 - val_loss: 0.9584 - val_accuracy: 0.5175
Epoch 5/12
553/553 [==============================] - 15s 27ms/step - loss: 0.8257 - accuracy: 0.6021 - val_loss: 0.9088 - val_accuracy: 0.5622
Epoch 6/12
553/553 [==============================] - 15s 28ms/step - loss: 0.7597 - accuracy: 0.6412 - val_loss: 0.9166 - val_accuracy: 0.5777
Epoch 7/12
553/553 [==============================] - 12s 21ms/step - loss: 0.6867 - accuracy: 0.6831 - val_loss: 0.8887 - val_accuracy:

In [ ]:
# try 2nd set of parameters
class_model = build_model(num_classes=4)

class_history = class_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=5,
    epochs=20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

In [ ]:
class_acc = class_model.evaluate(X_test, y_test, verbose=0)[1]
print("Test Accuracy (Class Model): {:.2f}%".format(class_acc * 100))

Test Accuracy (Class Model): 62.33%


In [ ]:
X_train, X_test, y_train, y_test = preprocess_inputs(data, target='User')


In [ ]:
X_train

,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,F7 delta m,...,AF4 delta m,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m,Class_0.0,Class_1.0,Class_2.0
0,0.136557,0.109199,-0.762146,-0.834782,-1.100221,-0.054621,0.025642,-0.592080,-0.503282,-0.238427,...,-0.477637,-0.453468,-0.459838,-0.381190,-0.451708,-0.508121,-0.532689,-0.707279,-0.711600,1.423587
1,0.055419,-0.097328,-0.187676,-0.538443,-0.824569,-0.541600,-0.763167,-0.789313,-1.258213,-1.347086,...,-0.574686,-0.603038,-0.413381,-0.459061,-0.446980,-0.526168,-0.517365,-0.707279,-0.711600,1.423587
2,0.103683,0.036032,0.507173,0.065498,0.415348,1.354333,1.190130,1.538453,1.379261,1.485966,...,0.026121,-0.610913,-0.367447,-0.306535,-0.213570,-0.494130,-0.463914,-0.707279,1.405284,-0.702451
3,-1.015328,-0.064868,0.323611,0.054294,0.796912,0.984926,1.053840,1.861944,1.499653,1.849122,...,-0.053724,-0.348525,-0.212232,-0.305695,-0.314834,-0.487564,-0.512801,-0.707279,1.405284,-0.702451
4,-0.239211,-0.186150,-0.515752,0.201501,1.746550,0.395262,-1.252509,-1.468086,-0.431629,-0.680551,...,-0.520612,-0.504640,-0.503606,-0.328398,-0.439712,-0.553785,-0.560519,-0.707279,-0.711600,1.423587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9211,-1.625647,-1.510879,-0.601444,-0.272073,-0.438529,0.602139,2.976946,0.754882,1.895693,2.055473,...,-1.261108,-0.543285,-0.422701,-0.470908,-0.452206,-0.495726,-0.449000,-0.707279,-0.711600,1.423587
9212,-1.064711,-1.203765,-0.348828,-0.398137,-0.074671,0.477677,1.374159,2.214734,1.298586,1.672208,...,-1.113318,-0.607755,-0.500974,-0.265585,-0.420859,-0.463723,-0.454791,-0.707279,1.405284,-0.702451
9213,-0.163182,-0.221880,-0.504956,0.542650,1.029620,-0.089580,-1.289843,-1.078598,-1.139330,-1.282820,...,-0.511175,-0.386812,-0.302039,-0.395470,-0.510833,-0.538754,-0.552641,-0.707279,-0.711600,1.423587
9214,-0.186939,-0.433750,0.894462,-0.172772,0.014341,0.055100,-0.553529,-0.989989,0.229215,-0.100149,...,-0.499725,-0.435756,-0.514580,-0.497519,-0.518822,-0.517426,-0.561628,1.413868,-0.711600,-0.702451


In [ ]:
y_train

10812    1
9395     3
1667     2
11267    2
5678     3
        ..
8324     2
10206    2
6253     3
10123    1
5600     1
Name: User, Length: 9216, dtype: int64

In [ ]:
y_train.value_counts()

2    2323
3    2321
0    2301
1    2271
Name: User, dtype: int64

In [ ]:
def build_model(num_classes=4):
    
    inputs = tf.keras.Input(shape=(None, 115))
    x = tf.keras.layers.Reshape((-1, 115))(inputs)
    x = tf.keras.layers.LSTM(256, return_sequences=True)(x)
    x = tf.keras.layers.LSTM(128, return_sequences=True)(x)
    x = tf.keras.layers.LSTM(64)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
user_model = build_model(num_classes=4)

user_history = user_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=20,
    epochs=20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/20
369/369 [==============================] - 60s 27ms/step - loss: 0.1817 - accuracy: 0.9394 - val_loss: 0.0240 - val_accuracy: 0.9935
Epoch 2/20
369/369 [==============================] - 8s 20ms/step - loss: 0.0192 - accuracy: 0.9940 - val_loss: 0.0072 - val_accuracy: 0.9984
Epoch 3/20
369/369 [==============================] - 8s 22ms/step - loss: 0.0071 - accuracy: 0.9978 - val_loss: 0.0039 - val_accuracy: 0.9995
Epoch 4/20
369/369 [==============================] - 8s 21ms/step - loss: 0.0054 - accuracy: 0.9982 - val_loss: 0.0026 - val_accuracy: 0.9995
Epoch 5/20
369/369 [==============================] - 9s 25ms/step - loss: 0.0074 - accuracy: 0.9980 - val_loss: 0.0040 - val_accuracy: 0.9984
Epoch 6/20
369/369 [==============================] - 9s 25ms/step - loss: 7.3325e-04 - accuracy: 1.0000 - val_loss: 0.0082 - val_accuracy: 0.9967
Epoch 7/20
369/369 [==============================] - 8s 22ms/step - loss: 1.7428e-04 - accuracy: 1.0000 - val_loss: 0.0025 - val_accurac

In [ ]:
class_acc = user_model.evaluate(X_test, y_test, verbose=0)[1]
print("Test Accuracy (Class Model): {:.2f}%".format(class_acc * 100))

Test Accuracy (Class Model): 99.87%


In [ ]:
user_model = build_model(num_classes=4)

user_history = user_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=5,
    epochs=20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/20
1475/1475 [==============================] - 25s 14ms/step - loss: 0.0943 - accuracy: 0.9695 - val_loss: 0.0144 - val_accuracy: 0.9951
Epoch 2/20
1475/1475 [==============================] - 20s 13ms/step - loss: 0.0161 - accuracy: 0.9950 - val_loss: 0.0159 - val_accuracy: 0.9935
Epoch 3/20
1475/1475 [==============================] - 21s 14ms/step - loss: 0.0111 - accuracy: 0.9963 - val_loss: 0.0194 - val_accuracy: 0.9951
Epoch 4/20
1475/1475 [==============================] - 19s 13ms/step - loss: 0.0047 - accuracy: 0.9988 - val_loss: 9.6223e-04 - val_accuracy: 1.0000
Epoch 5/20
1475/1475 [==============================] - 20s 14ms/step - loss: 0.0046 - accuracy: 0.9986 - val_loss: 8.2165e-04 - val_accuracy: 0.9995
Epoch 6/20
1475/1475 [==============================] - 20s 13ms/step - loss: 0.0076 - accuracy: 0.9980 - val_loss: 0.0055 - val_accuracy: 0.9967
Epoch 7/20
1475/1475 [==============================] - 21s 14ms/step - loss: 2.9329e-04 - accuracy: 1.0000 - val_lo

In [58]:
class_acc = user_model.evaluate(X_test, y_test, verbose=0)[1]
print("Test Accuracy (Class Model): {:.2f}%".format(class_acc * 100))

Test Accuracy (Class Model): 99.91%
